# **Data Collection & Storage — Final Project (Focused on Collection & Cleaning)**

This notebook is scoped to the first two lifecycle steps: **Data Collection** and **Data Cleaning**. The downstream analysis and communication will be completed later.

**Problematic (context for later analysis):**
- Which lines are the most well served for Parisians?
- What is the main cause of delays?
- How much does it differ from other lines?

We will only use the CSV file: `Regularities_by_liaisons_Trains_France.csv`.

---

## Principles We Follow
- Clarity and reproducibility first: every step is explained and deterministic.
- Decisions are justified using both observed data properties and data best practices.
- No ML/advanced stats here; we prepare a clean, reliable dataset for future analysis.

## 1. Data Collection

### 1.1 Dataset selection & scope
- Source: Kaggle dataset “Public transport traffic data in France” ([link](https://www.kaggle.com/datasets/gatandubuc/public-transport-traffic-data-in-france)).
- We explicitly use only one file from the dataset: `Regularities_by_liaisons_Trains_France.csv`.
- Rationale: this file contains service regularity information per liaison/line, which is directly relevant for understanding service levels and delays.

### 1.2 Provenance & reproducibility
- We retrieve data programmatically using `kagglehub` to guarantee a deterministic path and versioned download.
- We copy the CSV to a local `data/raw` folder to ensure a stable, project-local reference path for subsequent steps.
- Ethical note: data is public/open; we will cite data source in the final report and respect licensing.

### 1.3 Practical notes for this notebook
- We aim for clear, well-documented code; each decision is justified by what we observe in the data and by standard data-cleaning best practices.
- No analysis is performed here; the goal is to produce a clean, consistent, and well-typed dataset ready for downstream analysis.

In [1]:
from __future__ import annotations

import os
from pathlib import Path
import pandas as pd
import numpy as np
import kagglehub

# Display options for consistent review
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# 1) Deterministic download (versioned) via kagglehub
#    Decision: programmatic retrieval ensures reproducibility and avoids manual steps.
dataset_dir = Path(kagglehub.dataset_download("gatandubuc/public-transport-traffic-data-in-france"))
print("Downloaded dataset directory:", dataset_dir)

# 2) Select only the specified file
raw_filename = "Regularities_by_liaisons_Trains_France.csv"
source_csv_path = dataset_dir / raw_filename

# 3) Ensure a stable, project-local raw data path
project_root = Path.cwd()
raw_dir = project_root / "data" / "raw"
raw_dir.mkdir(parents=True, exist_ok=True)
local_raw_csv = raw_dir / raw_filename

if not source_csv_path.exists():
    # If the expected file is not found, list available files to aid debugging
    available = sorted([p.name for p in dataset_dir.rglob("*.csv")])
    print("Expected file not found. Available CSVs:", available)
    raise FileNotFoundError(f"Could not find {raw_filename} under {dataset_dir}")

# Copy only if needed to keep local reference stable
if not local_raw_csv.exists() or os.path.getsize(local_raw_csv) != os.path.getsize(source_csv_path):
    local_raw_csv.write_bytes(source_csv_path.read_bytes())

print("Local raw CSV:", local_raw_csv)

# 4) Load CSV (robust to common text encodings); we do not pre-guess dtypes to avoid silent coercions
#    Decision: use low_memory=False for consistent type inference; fallback encoding only if needed.
try:
    df_raw = pd.read_csv(local_raw_csv, low_memory=False)
except UnicodeDecodeError:
    df_raw = pd.read_csv(local_raw_csv, low_memory=False, encoding="latin1")

print("Raw shape:", df_raw.shape)
df_raw.head(3)

/Users/yassine/Desktop/Data Storage/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloaded dataset directory: /Users/yassine/.cache/kagglehub/datasets/gatandubuc/public-transport-traffic-data-in-france/versions/1
Local raw CSV: /Users/yassine/Desktop/Data Storage/data/raw/Regularities_by_liaisons_Trains_France.csv
Raw shape: (7806, 32)


,Year,Month,Departure station,Arrival station,Average travel time (min),Number of expected circulations,Number of cancelled trains,Number of late trains at departure,Average delay of late departing trains (min),Average delay of all departing trains (min),Comment (optional) delays at departure,Number of trains late on arrival,Average delay of late arriving trains (min),Average delay of all arriving trains (min),Comment (optional) delays on arrival,"% trains late due to external causes (weather, obstacles, suspicious packages, malevolence, social movements, etc.)","% trains late due to railway infrastructure (maintenance, works)","% trains late due to traffic management (rail line traffic, network interactions)",% trains late due to rolling stock,% trains late due to station management and reuse of material,"% trains late due to passenger traffic (affluence, PSH management, connections)",Number of late trains > 15min,Average train delay > 15min,Number of late trains > 30min,Number of late trains > 60min,Period,Delay due to external causes,Delay due to railway infrastructure,Delay due to traffic management,Delay due to rolling stock,Delay due to station management and reuse of material,Delay due to travellers taken into account
0,2019,7.0,ANGOULEME,PARIS MONTPARNASSE,131.914980,247.0,0.0,191.0,3.576353,2.678273,NaN,41.0,22.924797,5.233333,NaN,0.250000,0.150000,0.275000,0.125000,0.025000,0.175000,21.0,32.965873,7.0,2.0,2019-07,25.000000,15.000000,27.500000,12.500000,2.500000,17.500000
1,2019,7.0,PARIS MONTPARNASSE,LA ROCHELLE VILLE,175.611570,242.0,0.0,178.0,9.780805,7.033609,NaN,47.0,27.085106,6.123967,NaN,0.200000,0.244444,0.266667,0.244444,0.000000,0.044444,35.0,32.057143,14.0,2.0,2019-07,20.000000,24.444444,26.666667,24.444444,0.000000,4.444444
2,2019,7.0,LE MANS,PARIS MONTPARNASSE,62.395349,435.0,5.0,391.0,3.896974,3.529341,NaN,91.0,18.440659,4.430194,"Ce mois ci, la destination a été marquée par l...",0.161765,0.323529,0.264706,0.147059,0.029412,0.073529,30.0,42.367241,13.0,4.0,2019-07,16.176471,32.352941,26.470588,14.705882,2.941176,7.352941


## 2. Data Cleaning

Goal: transform the raw table into a clean, consistent, analysis-ready dataset.

We will:
- Inspect schema, types, and missingness to understand the data.
- Define and justify a cleaning plan based on observations and best practices.
- Implement a transparent, deterministic cleaning pipeline that can be re-run.


In [2]:
# 2.1 Inspect raw structure
print("Raw shape (rows, cols):", df_raw.shape)
print("\nColumn names (original):\n", list(df_raw.columns))

print("\nData types:")
df_raw.dtypes.to_frame("dtype").T

print("\nInfo:")
df_raw.info()

print("\nSample records:")
df_raw.head(5)

print("\nMissingness (top 20 by fraction):")
missing_fraction = df_raw.isna().mean().sort_values(ascending=False)
missing_fraction.head(20)

Raw shape (rows, cols): (7806, 32)

Column names (original):
 ['Year', 'Month', 'Departure station', 'Arrival station', 'Average travel time (min)', 'Number of expected circulations', 'Number of cancelled trains', 'Number of late trains at departure', 'Average delay of late departing trains (min)', 'Average delay of all departing trains (min)', 'Comment (optional) delays at departure', 'Number of trains late on arrival', 'Average delay of late arriving trains (min)', 'Average delay of all arriving trains (min)', 'Comment (optional) delays on arrival', '% trains late due to external causes (weather, obstacles, suspicious packages, malevolence, social movements, etc.)', '% trains late due to railway infrastructure (maintenance, works)', '% trains late due to traffic management (rail line traffic, network interactions)', '% trains late due to rolling stock', '% trains late due to station management and reuse of material', '% trains late due to passenger traffic (affluence, PSH management,

Comment (optional) delays at departure                                                                                 1.000000
Comment (optional) delays on arrival                                                                                   0.769024
% trains late due to railway infrastructure (maintenance, works)                                                       0.034717
Delay due to station management and reuse of material                                                                  0.034717
Delay due to rolling stock                                                                                             0.034717
Delay due to traffic management                                                                                        0.034717
Delay due to railway infrastructure                                                                                    0.034717
Delay due to external causes                                                                            

### 2.2 Cleaning plan & decisions

Based on inspection and data-cleaning best practices, we will:

1) Standardize column names to `snake_case` for consistency and easier downstream use.
2) Remove exact duplicate rows to avoid double-counting.
3) Trim whitespace from textual columns and normalize casing where appropriate.
4) Parse date-like columns (e.g., containing `date`/`jour`) to datetime for reliable time operations.
5) Convert numeric-like columns currently stored as text to numeric with safe coercion.
6) Handle missing values:
   - Drop rows missing essential identifiers (e.g., liaison/line/date) if present.
   - For count-like fields (e.g., number of delays/canceled services), impute 0 only when semantically sound.
   - For categorical reason fields, set missing to `Unknown` and normalize labels.
7) Persist a clean dataset to `data/processed` (CSV and Parquet if available) for reproducibility.

We document each decision inline below and rely on column-name heuristics when the schema varies (robustness without hard-coding unknowns).

In [3]:
# 2.3 Implement cleaning pipeline
import re
from typing import Iterable

# Helper: normalize to snake_case and ASCII-friendly
# Decision: consistent naming simplifies downstream grouping/joins and avoids locale-specific issues.
def normalize_col(name: str) -> str:
    s = name.strip().lower()
    # Replace separators with underscore
    s = re.sub(r"[\s\-/]+", "_", s)
    # Remove any remaining non-alnum/underscore
    s = re.sub(r"[^0-9a-zA-Z_]+", "", s)
    # Collapse multiple underscores
    s = re.sub(r"_+", "_", s).strip("_")
    return s

# Start from a deep copy of raw
df_clean = df_raw.copy(deep=True)
df_clean.columns = [normalize_col(c) for c in df_clean.columns]

# Remove exact duplicates before any further mutation (prevents double-count)
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
removed_dups = initial_rows - len(df_clean)

# Trim whitespace in all object columns
obj_cols = [c for c in df_clean.columns if df_clean[c].dtype == "object"]
for c in obj_cols:
    df_clean[c] = df_clean[c].astype(str).str.strip()

# Identify date-like columns (heuristic by name)
date_like_cols = [c for c in df_clean.columns if re.search(r"\b(date|jour|calendar|perio)\b", c)]
for c in date_like_cols:
    df_clean[c] = pd.to_datetime(df_clean[c], errors="coerce", dayfirst=True, infer_datetime_format=True)

# Identify numeric-like columns by name patterns
numeric_name_patterns = [
    r"^(nb|nombre|count|total|vol|qty|quant|qte)",
    r"(delai|retard|late|min|sec|heures?)",
    r"(annul|cancel)",
    r"(train|service|trajet|travel)s?",
]

maybe_numeric_cols: list[str] = []
for c in df_clean.columns:
    if any(re.search(pat, c) for pat in numeric_name_patterns):
        maybe_numeric_cols.append(c)

# Additionally include columns that look numeric but are objects
maybe_numeric_cols += [
    c for c in df_clean.columns
    if df_clean[c].dtype == "object" and df_clean[c].str.match(r"^[-+]?\d+[\d.,]*$", na=False).mean() > 0.5
]
maybe_numeric_cols = sorted(set(maybe_numeric_cols))

# Coerce to numeric safely (commas to dots; non-numeric -> NaN)
for c in maybe_numeric_cols:
    if df_clean[c].dtype == "object":
        df_clean[c] = df_clean[c].str.replace(",", ".", regex=False)
    df_clean[c] = pd.to_numeric(df_clean[c], errors="coerce")

# Identify potential essential identifier columns (liaison/line/date)
essential_keywords = ["liaison", "ligne", "line", "route", "relation", "code", "id", "date", "jour"]
essential_cols = [c for c in df_clean.columns if any(k in c for k in essential_keywords)]

# Decision: only drop rows if we have at least one credible identifier present; else keep all (conservative)
if essential_cols:
    before_drop = len(df_clean)
    df_clean = df_clean.dropna(subset=essential_cols, how="any")
    dropped_for_missing_keys = before_drop - len(df_clean)
else:
    dropped_for_missing_keys = 0

# Normalize reason/motif columns for delays/cancellations
reason_keywords = ["motif", "cause", "reason"]
reason_cols = [c for c in df_clean.columns if any(k in c for k in reason_keywords)]
for c in reason_cols:
    df_clean[c] = df_clean[c].fillna("Unknown").astype(str).str.strip().str.replace("\s+", " ", regex=True).str.title()

# Impute zeros for count-like columns (only when semantically appropriate)
count_like_patterns = [r"^(nb|nombre|count|total)", r"(annul|cancel)", r"(retard|delay|delai)"]
count_like_cols = [c for c in df_clean.columns if any(re.search(pat, c) for pat in count_like_patterns)]
for c in count_like_cols:
    if pd.api.types.is_numeric_dtype(df_clean[c]):
        df_clean[c] = df_clean[c].fillna(0)

# Final light tidy: reorder columns (ids first, then dates, then others)
id_first = [c for c in df_clean.columns if re.search(r"\b(id|code|ligne|line|liaison)\b", c)]
date_first = [c for c in df_clean.columns if c in date_like_cols]
other_cols = [c for c in df_clean.columns if c not in set(id_first + date_first)]
df_clean = df_clean[id_first + date_first + other_cols]

print({
    "initial_rows": int(initial_rows),
    "removed_duplicates": int(removed_dups),
    "dropped_for_missing_keys": int(dropped_for_missing_keys),
    "final_rows": int(len(df_clean)),
    "final_cols": int(df_clean.shape[1]),
})
df_clean.head(5)


{'initial_rows': 7806, 'removed_duplicates': 0, 'dropped_for_missing_keys': 271, 'final_rows': 7535, 'final_cols': 32}


<>:78: SyntaxWarning: invalid escape sequence '\s'
<>:78: SyntaxWarning: invalid escape sequence '\s'
/var/folders/4c/27024g9s33d5p6whgp0356km0000gn/T/ipykernel_20257/2887639909.py:78: SyntaxWarning: invalid escape sequence '\s'
  df_clean[c] = df_clean[c].fillna("Unknown").astype(str).str.strip().str.replace("\s+", " ", regex=True).str.title()


,year,month,departure_station,arrival_station,average_travel_time_min,number_of_expected_circulations,number_of_cancelled_trains,number_of_late_trains_at_departure,average_delay_of_late_departing_trains_min,average_delay_of_all_departing_trains_min,comment_optional_delays_at_departure,number_of_trains_late_on_arrival,average_delay_of_late_arriving_trains_min,average_delay_of_all_arriving_trains_min,comment_optional_delays_on_arrival,trains_late_due_to_external_causes_weather_obstacles_suspicious_packages_malevolence_social_movements_etc,trains_late_due_to_railway_infrastructure_maintenance_works,trains_late_due_to_traffic_management_rail_line_traffic_network_interactions,trains_late_due_to_rolling_stock,trains_late_due_to_station_management_and_reuse_of_material,trains_late_due_to_passenger_traffic_affluence_psh_management_connections,number_of_late_trains_15min,average_train_delay_15min,number_of_late_trains_30min,number_of_late_trains_60min,period,delay_due_to_external_causes,delay_due_to_railway_infrastructure,delay_due_to_traffic_management,delay_due_to_rolling_stock,delay_due_to_station_management_and_reuse_of_material,delay_due_to_travellers_taken_into_account
0,2019,7.0,ANGOULEME,PARIS MONTPARNASSE,131.914980,247.0,0.0,191.0,3.576353,2.678273,0.0,41.0,22.924797,5.233333,nan,0.25,0.150000,0.275000,0.125000,0.025000,0.175000,21.0,32.965873,7.0,2.0,2019-07,25.0,15.000000,27.500000,12.500000,2.500000,17.500000
1,2019,7.0,PARIS MONTPARNASSE,LA ROCHELLE VILLE,175.611570,242.0,0.0,178.0,9.780805,7.033609,0.0,47.0,27.085106,6.123967,nan,0.2,0.244444,0.266667,0.244444,0.000000,0.044444,35.0,32.057143,14.0,2.0,2019-07,20.0,24.444444,26.666667,24.444444,0.000000,4.444444
2,2019,7.0,LE MANS,PARIS MONTPARNASSE,62.395349,435.0,5.0,391.0,3.896974,3.529341,0.0,91.0,18.440659,4.430194,"Ce mois ci, la destination a été marquée par l...",0.1617647058819999,0.323529,0.264706,0.147059,0.029412,0.073529,30.0,42.367241,13.0,4.0,2019-07,16.1764705882,32.352941,26.470588,14.705882,2.941176,7.352941
3,2019,7.0,ST MALO,PARIS MONTPARNASSE,172.421053,114.0,0.0,101.0,1.950990,1.685673,0.0,15.0,20.618889,3.812135,nan,0.153846153846,0.153846,0.230769,0.384615,0.000000,0.076923,8.0,27.620833,2.0,0.0,2019-07,15.3846153846,15.384615,23.076923,38.461538,0.000000,7.692308
4,2019,7.0,PARIS MONTPARNASSE,ST PIERRE DES CORPS,67.310000,404.0,4.0,284.0,8.379108,5.803125,0.0,73.0,19.608447,4.176208,nan,0.184615384615,0.123077,0.400000,0.169231,0.076923,0.046154,28.0,37.658333,12.0,3.0,2019-07,18.4615384615,12.307692,40.000000,16.923077,7.692308,4.615385


In [4]:
# 2.4 Persist cleaned dataset for reproducibility
processed_dir = project_root / "data" / "processed"
processed_dir.mkdir(parents=True, exist_ok=True)

clean_csv_path = processed_dir / "regularities_by_liaisons_trains_france_clean.csv"
df_clean.to_csv(clean_csv_path, index=False)

# Parquet is efficient for downstream analytics if available
try:
    clean_parquet_path = processed_dir / "regularities_by_liaisons_trains_france_clean.parquet"
    df_clean.to_parquet(clean_parquet_path, index=False)
    wrote_parquet = True
except Exception as e:
    wrote_parquet = False

print({
    "clean_csv_path": str(clean_csv_path),
    "wrote_parquet": wrote_parquet,
    "processed_dir": str(processed_dir),
})

{'clean_csv_path': '/Users/yassine/Desktop/Data Storage/data/processed/regularities_by_liaisons_trains_france_clean.csv', 'wrote_parquet': False, 'processed_dir': '/Users/yassine/Desktop/Data Storage/data/processed'}
